In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install --target=$nb_path torch-scatter
!pip install --target=$nb_path torch-sparse
!pip install --target=$nb_path torch-geometric

  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=282114 sha256=9aefe95a0b18843e8d90cea87f16be19c198123d15016ba4cd3658dfa85f873b
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter
     |████████████████████████████████| 43 kB 1.2 MB/s 
  Created wheel for torch-sparse: filename=torch_sparse-0.6.12-cp37-cp37m-linux_x86_64.whl size=505740 sha256=51580b730495750c940ba8dda5344b3bd701132b606b34940af82f1fc31607aa
  Stored in directory: /root/.cache/pip/wheels/fb/e2/2f/44956c61e3299573ffe12da9d1374c7576ca0c5fb1fe1ed38c
Successfully built torch-sparse
     |████████████████████████████████| 325 kB 5.0 MB/s 
     |████████████████████████████████| 407 kB 30.8 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.2-py3-none-any.whl size=535570 sha256=da8a9cbdb88447f6e6b05ccdb3f364c59739e2a

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv,GATConv

In [ ]:
!pip install --target=$nb_path torch-scatter
!pip install --target=$nb_path torch-sparse
!pip install --target=$nb_path torch-geometric

import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv,GATConv


class DGNN(nn.Module):
  def __init__(self, feature, hidden, output, nodes, window, dropout, hist=7):
    super(DGNN, self).__init__()

    self.state = 50
    self.feature = feature
    self.hidden = hidden
    self.nodes = nodes
    self.window = window
    self.hist = hist

    # neural network layers
    self.conv1 = GATConv(feature, hidden, concat=False)
    self.conv2 = GATConv(hidden, hidden, concat=False)
    self.bn1 = nn.BatchNorm1d(hidden)
    self.bn2 = nn.BatchNorm1d(hidden)
    self.rnn1 = nn.GRU(2*hidden, hidden, 1)
    self.rnn2 = nn.GRU(hidden, hidden, 1)
    self.fc1 = nn.linear(2*hidden + window*hist, hidden)
    self.fc2 = nn.linear(hidden, output)
    self.dropout = nn.Dropout(dropout)
    self.relu = nn.ReLU()
    self.w1 = nn.parameter.Parameter(torch.Tensor(nhid, 1))
    self.w2 = nn.parameter.Parameter(torch.Tensor(nhid, 1))
    self.m = nn.Softmax(dim=0)

  def forward(self, adj, x):
    lst = list()
    weight = adj.coalesce().values()
    adj = adj.coalesce().indices()
    skip = x.view(-1, self.window, self.nodes, self.feature)
    skip = skip[:,:,:self.state,:]
    skip = torch.transpose(skip,1,2).reshape(-1,self.window,self.feature)
    
    x, (edge, att) = self.conv1(x,adj,reshape_attenuation_weights=True)
    x = self.dropout(self.bn1(self.relu(x)))
    x2 = x.reshape(-1, self.hidden)
    x2 = x2[:,:self.nodes,self.hidden]
    x2 = x2.reshape(-1, self.hidden)
    lst.append(x2)

    x = torch.cat(lst, dim=1)
    x = x.view(-1, self.window, self.state, x.size(1))
    x = torch.transpose(x, 0, 1)
    x = x.contiguous().view(self.window, -1, x.size(3))
    out1,hn1 = self.rnn1(x)
    out2,hn2 = self.rnn2(out1)
    x = torch.cat([hn1[0,:,:],hn2[0,:,:]], dim=1)
    
    skip = skip.reshape(skip, size(0), -1)
    x = torch.cat([x,skip], dim=1)
    x = self.relu(self.fc1(x))
    x = self.dropout(x)
    x = self.relu(self.fc2(x)).squeeze()
    x = x.view(-1)
    return x